In [3]:
import pandas as pd
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [35]:
# 80000row
user_train = pd.read_csv('ml-100k_GRS/u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]

## u1.base 각 유저가 각 아이템에 준 각 별점


# 100000row
user_total = pd.read_csv('ml-100k_GRS/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_total = user_total[['userID','itemID','rating']]

# 20000row
user_test = pd.read_csv('ml-100k_GRS/u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]


## 943명이 1682개를 1~5로 평균냄

In [36]:
user_total

,userID,itemID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [29]:

# 0 ~ 1사이의 범위로 평점 데이터를 표준화 min:1 max:5
'''
min_max_scaler = MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(user_train[['rating']])
user_train[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_total[['rating']])
user_total[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_test [['rating']])
user_test [['rating']] = x_scaled
'''

In [30]:
#user_total
# 0, 25, 50, 75, 1, 의 값으로 변환된다.

,userID,itemID,rating
0,196,242,0.50
1,186,302,0.50
2,22,377,0.00
3,244,51,0.25
4,166,346,0.00
...,...,...,...
99995,880,476,0.50
99996,716,204,1.00
99997,276,1090,0.00
99998,13,225,0.25


In [37]:
user_item_total = user_total.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0) # fillna 를 하지 않을 경우 NaN 값 들어감
user_item_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_item_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

In [38]:
user_item_total

itemID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


각 유저가 각 아이템들에 대해 어떻게 평점을 주었는지를 한 눈에 볼 수 있다.

In [20]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from itertools import cycle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ndcg_score

#cluster용 dataset
SVD = TruncatedSVD(n_components=2) # 상위 n개의 특잇값을 추출해 n차원으로 생성 / 지금은 2차원

SVD 로 보지 않은 영화 평점을 예측하는 방법 : https://walwalgabu.tistory.com/entry/1-Numpy%EB%A7%8C-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-SVD-%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%EC%9D%84-%EA%B0%84%EB%8B%A8%ED%9E%88-%EC%8B%A4%ED%97%98%ED%95%B4%EB%B3%B4%EC%9E%90Truncated-SVD  

특히 truncated SVD 는 그 SVD 를 통해 일궈낸 특이값들 중 상위 N 개만을 추출한 것을 의미한다.

In [34]:
full_rank_item_total = SVD.fit_transform(user_item_total)
full_rank_item_total = pd.DataFrame(full_rank_item_total)
full_rank_item_total

,0,1
0,42.156451,1.462856
1,8.982352,-11.415710
2,3.624693,-6.272333
3,3.839383,-5.067712
4,20.979029,2.242444
...,...,...
938,7.227809,-10.082025
939,19.223048,-2.063405
940,4.769211,-6.126077
941,15.395186,1.982231


truncated SVD 로 두 개의 component(특잇값) 을 추출
이를 가지고 내적을 통해 다시 원래의 값을 반환

In [45]:
km = KMeans(n_clusters=5, init='k-means++')
cluster = km.fit(user_item_total)

cluster_id = pd.DataFrame(cluster.labels_)
cluster_id

,0
0,1
1,0
2,4
3,4
4,3
...,...
938,0
939,3
940,4
941,3


In [46]:
cluster_id.index

RangeIndex(start=0, stop=943, step=1)

In [49]:
cluster_id.index = user_item_total.index
cluster_id

,0
userID,
1,1
2,0
3,4
4,4
5,3
...,...
939,0
940,3
941,4


## NDCG

In [53]:
def groupRS_ndcg(num, total_matrix, train_matrix):
    total_matrix  # user_item_total 
    train_matrix  # user_item_train
    test_matrix = user_item_test #user_item_test
    
    
    ## 1. K-means로 그룹 clustering(total 대상) #아직 SVD 로 평점을 예측하지 않은 상태

    km = KMeans(n_clusters=num, init='k-means++')
    cluster = km.fit(total_matrix)
    cluster_id = pd.DataFrame(cluster.labels_) # 모든  user의 클러스터 n 이 표시됨

    cluster_id.index = total_matrix.index # userId 칼럼을 인덱스로 설정 1~943
    cluster_id.rename(columns = {0 : 'cluster'}, inplace = True) # 모든  user의 클러스터
    
    length = [0]*num
    for i in range(num):
        length[i] = len(cluster_id[cluster_id.cluster==i]) # 각 클러스터에 해당하는 개수
    
    print("각 cluster 별 user 회원 수 : ", length)

    # 2. train, test 에 cluster 정보 추가
    user_item_train_cl = pd.concat([train_matrix, cluster_id], axis=1, join='inner')
    user_item_test_cl = pd.concat([test_matrix, cluster_id], axis=1, join='inner')
    






    ## 3. (train 대상) 클러스터 별로 각 item의 mean 값 구함 
    mean_rating = pd.DataFrame(columns = user_item_train_cl.columns)
    mean_rating.set_index('cluster') # cluster 를 인덱스로 설정하고 이걸 기준으로 각 아이템 당 모든 유저의 별점 평균 구할 예정
    
    
    for i in range(num):
        mean_rating = mean_rating.append(user_item_train_cl[user_item_train_cl.cluster == i].mean(axis=0), ignore_index=True) # 각 클러스터별 평균 계산해서 추가
    
    mean_rating = mean_rating.set_index('cluster')
    mean_rating
    
    #print(mean_rating)


    ## 3. train-test set의 columns(item id) 맞추기 (miss matching 제거)
    for c in user_item_train_cl.columns:
        if c not in user_item_test_cl.columns:
            del mean_rating[c]
        
    for c in user_item_test_cl.columns:
        if c not in user_item_train_cl.columns:
            del user_item_test_cl[c]
            
    y_pred = mean_rating 
    y_true = user_item_test_cl
    
    
    result = [0]*num # 결과값 저장 리스트
    



    
    ## 4. 각 결과 값에 nDCG 더해줌
    for idx in test_matrix.index:
        cluster_num = int(y_true.loc[idx].cluster)
        result[cluster_num] += ndcg_score([y_true.loc[idx][:-1]], [y_pred.loc[cluster_num]])
        #result[cluster] += ndcg_score([user_item_test_cl.loc[idx][:-1]], [mean_rating.loc[cluster]], k=4)
    
    
    ## 5. 최종적으로 각 nDCG값 / 각 cluster의 요소 개수
    for i in range(num):
        result[i] = result[i]/length[i]
    
    print("cluster수:",len(length),"/ NDCG:",sum(result)/len(length))  
    print('%.5f'%(sum(result)/(len(length))))

In [54]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

cluster_num = range(2,51)
#cluster_num=[2]

for i in cluster_num:
    #reduced_total = SVD.fit_transform(user_item_total)
    #reduced_total = pd.DataFrame(reduced_total)
    #reduced_total.index = total_user_idx
    
    #user_item_train = pd.DataFrame(user_item_train)
    
    groupRS_ndcg(i, user_item_total, user_item_train)

각 cluster 별 user 회원 수 :  [644, 299]
cluster수: 2 / NDCG: 0.2687740005034088
0.26877
각 cluster 별 user 회원 수 :  [268, 545, 130]
cluster수: 3 / NDCG: 0.2915053221641824
0.29151
각 cluster 별 user 회원 수 :  [97, 539, 63, 244]
cluster수: 4 / NDCG: 0.3089775354579192
0.30898
각 cluster 별 user 회원 수 :  [230, 96, 167, 387, 63]
cluster수: 5 / NDCG: 0.29553927902061605
0.29554
각 cluster 별 user 회원 수 :  [171, 367, 51, 142, 163, 49]
cluster수: 6 / NDCG: 0.3042550243353891
0.30426
각 cluster 별 user 회원 수 :  [176, 96, 64, 23, 186, 102, 296]
cluster수: 7 / NDCG: 0.3057194049741872
0.30572
각 cluster 별 user 회원 수 :  [117, 53, 116, 85, 179, 53, 238, 102]
cluster수: 8 / NDCG: 0.29636020864766427
0.29636
각 cluster 별 user 회원 수 :  [22, 321, 57, 196, 173, 25, 47, 31, 71]
cluster수: 9 / NDCG: 0.3221906878567271
0.32219
각 cluster 별 user 회원 수 :  [71, 230, 36, 60, 182, 70, 114, 43, 79, 58]
cluster수: 10 / NDCG: 0.2978449527736864
0.29784
각 cluster 별 user 회원 수 :  [23, 75, 29, 63, 94, 54, 124, 101, 164, 186, 30]
cluster수: 11 / NDCG: 

In [23]:
## precision & recall : binary (2.5 이상 1, 나머지 0)
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def groupRS_pre_rec(num, total_matrix, train_matrix):
    total_matrix  # user_item_total 
    train_matrix  # user_item_train
    test_matrix = user_item_test
    
    
    ## 1. K-means로 그룹 clustering(total 대상)
    km = KMeans(n_clusters=num, init='k-means++')
    cluster = km.fit(total_matrix)
    cluster_id = pd.DataFrame(cluster.labels_) # 모든  user의 클러스터

    cluster_id.index = total_matrix.index
    cluster_id.rename(columns = {0 : 'cluster'}, inplace = True) # 모든  user의 클러스터
    
    length = [0]*num
    for i in range(num):
        length[i] = len(cluster_id[cluster_id.cluster==i]) # 각 클러스터에 해당하는 개수

    # train, test 에 cluster 정보 추가
    user_item_train_cl = pd.concat([train_matrix, cluster_id], axis=1, join='inner')
    user_item_test_cl = pd.concat([test_matrix, cluster_id], axis=1, join='inner')
    
    
    ## 2. 클러스터 별로 각 item의 mean 값 구함 (train 대상)
    mean_rating = pd.DataFrame(columns = user_item_train_cl.columns)
    mean_rating.set_index('cluster')
    
    for i in range(num):
        mean_rating = mean_rating.append(user_item_train_cl[user_item_train_cl.cluster == i].mean(axis=0), ignore_index=True)
    
    mean_rating = mean_rating.set_index('cluster')
    mean_rating
    
    
    ## 3. train-test set의 columns(item id) 맞추기 (miss matching 제거)
    for c in user_item_train_cl.columns:
        if c not in user_item_test_cl.columns:
            del mean_rating[c]
        
    for c in user_item_test_cl.columns:
        if c not in user_item_train_cl.columns:
            del user_item_test_cl[c] 
            
    y_pred = mean_rating 
    y_true = user_item_test_cl
    
    ## binary로 만들기 위해 2개의 분류로 생성
    y_pred[y_pred< 2.5] = 0
    y_pred[y_pred >= 2.5] = 1
    
    y_true[y_true< 2.5] = 0
    y_true[y_true >= 2.5] = 1
    
    pre_result = [0]*num # 결과값 저장 리스트
    rec_result = [0]*num
    
    ## 4. 각 결과 값에 precision, recall더해줌
    for idx in test_matrix.index:
        cluster_num = int(y_true.loc[idx].cluster)
        pre_result[cluster_num] += precision_score(list(y_true.loc[idx][:-1].values) ,list(y_pred.loc[cluster_num].values ), average = 'binary')
        rec_result[cluster_num] += recall_score(list(y_true.loc[idx][:-1].values) ,list(y_pred.loc[cluster_num].values ), average = 'binary')
    
    print(pre_result)
    print(rec_result)

    
    #print(sum(pre_result)/num)
    
    #print(sum(rec_result)/num)
    ## 5. 최종적으로 각 precision, recall값 / 각 cluster의 요소 개수
    #for i in range(num):
    #    result[i] = result[i]/length[i]

    
    #print("cluster수:",len(length),"/ NDCG:",sum(result)/len(length))  
    print('%.5f'%(sum(pre_result)/num), ", ",  '%.5f'%(sum(rec_result)/num))

In [24]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

cluster_num = range(2,31)
#cluster_num=[2]

for i in cluster_num:
    #reduced_total = SVD.fit_transform(user_item_total)
    #reduced_total = pd.DataFrame(reduced_total)
    #reduced_total.index = total_user_idx
    
    #user_item_train = pd.DataFrame(user_item_train)
    
    groupRS_pre_rec(i, user_item_total, user_item_train)

[0.0, 0]
[0.0, 0]
0.00000 ,  0.00000
[83.5, 0, 0]
[6.188595621403521, 0, 0]
27.83333 ,  2.06287
[48.099999999999945, 0.0, 0, 0]
[58.21009893633862, 0.0, 0, 0]
12.02500 ,  14.55252
[42.5, 33.846153846153804, 0, 0, 0]
[8.961068223715742, 30.51383311862477, 0, 0, 0]
15.26923 ,  7.89498
[0.0, 31.836363636363643, 0, 0, 0, 0]
[0.0, 39.81598149964606, 0, 0, 0, 0]
5.30606 ,  6.63600
[22.250000000000032, 34.0, 0, 0, 0, 0, 0]
[21.12491031814811, 1.2871803941353825, 0, 0, 0, 0, 0]
8.03571 ,  3.20173
[14.409090909090917, 55.400000000000105, 0, 0, 0, 0, 0, 0]
[13.43874436525986, 11.119365128679984, 0, 0, 0, 0, 0, 0]
8.72614 ,  3.06976
[29.100000000000016, 56.0, 0, 0, 0, 0, 0, 0, 0]
[9.957346303814557, 2.4919939951587113, 0, 0, 0, 0, 0, 0, 0]
9.45556 ,  1.38326
[0.0, 31.159999999999936, 0, 0, 0, 0, 0, 0, 0, 0]
[0.0, 19.23785554873005, 0, 0, 0, 0, 0, 0, 0, 0]
3.11600 ,  1.92379
[13.66666666666666, 38.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[8.113554139870692, 1.3248415595945782, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4.69

In [25]:
## precision & recall : multiclass(반올림)
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import math

def groupRS_pre_rec2(num, total_matrix, train_matrix):
    total_matrix  # user_item_total 
    train_matrix  # user_item_train
    test_matrix = user_item_test
    
    
    ## 1. K-means로 그룹 clustering(total 대상)
    km = KMeans(n_clusters=num, init='k-means++')
    cluster = km.fit(total_matrix)
    cluster_id = pd.DataFrame(cluster.labels_) # 모든  user의 클러스터

    cluster_id.index = total_matrix.index
    cluster_id.rename(columns = {0 : 'cluster'}, inplace = True) # 모든  user의 클러스터
    
    length = [0]*num
    for i in range(num):
        length[i] = len(cluster_id[cluster_id.cluster==i]) # 각 클러스터에 해당하는 개수

    # train, test 에 cluster 정보 추가
    user_item_train_cl = pd.concat([train_matrix, cluster_id], axis=1, join='inner')
    user_item_test_cl = pd.concat([test_matrix, cluster_id], axis=1, join='inner')
    
    
    ## 2. 클러스터 별로 각 item의 mean 값 구함 (train 대상)
    mean_rating = pd.DataFrame(columns = user_item_train_cl.columns)
    mean_rating.set_index('cluster')
    
    for i in range(num):
        mean_rating = mean_rating.append(user_item_train_cl[user_item_train_cl.cluster == i].mean(axis=0), ignore_index=True)
    
    mean_rating = mean_rating.set_index('cluster')
    mean_rating
    
    ## multiuclass 1~5점으로 반올림하여 다중 분류 생성
    for i in mean_rating.index:
        for j in mean_rating.columns:
             mean_rating.loc[i][j] = round(mean_rating.loc[i][j])

    user_item_test_cl = user_item_test_cl.round(0)
    

    ## 3. train-test set의 columns(item id) 맞추기 (miss matching 제거)
    for c in user_item_train_cl.columns:
        if c not in user_item_test_cl.columns:
            del mean_rating[c]
        
    for c in user_item_test_cl.columns:
        if c not in user_item_train_cl.columns:
            del user_item_test_cl[c] 
            
    y_pred = mean_rating 
    y_true = user_item_test_cl
    
    
    pre_result = [0]*num # 결과값 저장 리스트
    rec_result = [0]*num
    
    ## 4. 각 결과 값에 nDCG 더해줌
    for idx in test_matrix.index:
        cluster_num = int(y_true.loc[idx].cluster)
        pre_result[cluster_num] += precision_score(list(y_true.loc[idx][:-1].values) ,list(y_pred.loc[cluster_num].values ), average = 'macro')
        rec_result[cluster_num] += recall_score(list(y_true.loc[idx][:-1].values) ,list(y_pred.loc[cluster_num].values ), average = 'macro')
    
    print(pre_result)
    print(rec_result)
    #print(sum(pre_result)/num)
    
    #print(sum(rec_result)/num)
    ## 5. 최종적으로 각 precision, recall값 / 각 cluster의 요소 개수
    #for i in range(num):
    #    result[i] = result[i]/length[i]

    
    #print("cluster수:",len(length),"/ NDCG:",sum(result)/len(length))  
    print('%.5f'%(sum(pre_result)/num), ", ",  '%.5f'%(sum(rec_result)/num))

In [26]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

cluster_num = range(2,31)
#cluster_num=[2]

for i in cluster_num:
    #reduced_total = SVD.fit_transform(user_item_total)
    #reduced_total = pd.DataFrame(reduced_total)
    #reduced_total.index = total_user_idx
    
    #user_item_train = pd.DataFrame(user_item_train)
    
    groupRS_pre_rec2(i, user_item_total, user_item_train)

[26.648254764559113, 53.51832913719378]
[29.16322544764736, 56.06715694210131]
40.08329 ,  42.61519
[45.342621684601866, 21.91799567350174, 14.543358408169585]
[47.50794531709458, 23.90785094102233, 14.113544564426824]
27.26799 ,  28.50978
[19.97162242548278, 44.70255236192564, 9.492550939068396, 7.747905263638022]
[21.965960151654173, 46.86904744054555, 10.085381813708347, 7.4913701863737305]
20.47866 ,  21.60294
[7.747905263638022, 13.055853123213346, 33.048720187829474, 9.492550939068396, 19.14266577679049]
[7.4913701863737305, 15.34362568082785, 34.2585148173619, 10.085381813708347, 20.67276362932458]
16.49754 ,  17.57033
[13.794373953652293, 6.140231341218875, 13.55817032501908, 31.5488764044944, 11.804948193272935, 6.390321659577537]
[16.028968063000352, 7.04895190104157, 14.1565957028652, 32.87071932521023, 12.088443440823738, 6.32377873387311]
13.87282 ,  14.75291
[17.91588432984379, 6.618136353471958, 14.265387973195777, 9.02599605692279, 5.744657364598521, 9.834082621521315, 

y_true = [0, 1, 2, 0, 1, 2] y_pred = [0, 2, 1, 0, 0, 1]

y_true: target의 실제 값 (여기서는 test 값)

y_pred: 분류된 예상값 (여기서는 mean rating(train 값))